## Hosting und Interface bestehendes Modell

In der heutigen Übung möchten wir das Modell von letztem Mal (Handschrifterkennung) hosten und über eine REST-API zur Verfügung stellen. Dafür nutzen wir die Flask Library:

In [7]:
pip install flask

Note: you may need to restart the kernel to use updated packages.


### Laden des Modells inkl. Datenvorverarbeitung

Letztes Mal haben wir ein Modell auf Basis des MNIST Datensatzes von Kaggle heruntergeladen und mit Tensorflow geladen. Das machen wir heute genauso:

In [8]:
import tensorflow as tf
import os

model_path = "data/models/adhul000/mnist-99/tensorFlow2/default/1"

model = tf.keras.models.load_model(os.path.join(model_path, "best_model.h5"))

Nun benötigen wir noch unsere Funktionen zur Datenvorverarbeitung, die wir auch aus der letzten Übung kopieren können:

In [ ]:
import numpy as np
from PIL import Image

def preprocess_image(image_path) -> Image:
    img = Image.open(image_path)  
    img = img.convert("L")      # Schritt 1: Convert to grayscale
    img = img.resize((28, 28))  # Schritt 2: Resize to 28x28
    return img

In [ ]:
def predict(img) -> int:
    preprocessed_img = preprocess_image(img)
    np_img = np.array(preprocessed_img)      # Schritt 1: Typecast zu Numpy Array
    np_img = np_img / 255.0                  # Schritt 2: Normalisieren
    np_img = np_img.reshape((1, 28, 28, 1))  # Schritt 3: Reshape
    predictions = model.predict(np_img)      # Schritt 4: Vorhersage
    predicted_class = int(np.argmax(predictions[0]))
    return predicted_class

### REST-API Erstellen

Nun geht es an die eigentliche REST-API :). Lesen Sie sich den Artikel im Flask Tutorial "A Minimal Application" durch. Analog zu diesem Artikel implementieren Sie zuerst eine Route ```GET /version``` die uns Informationen über das Modell zur Verfügung stellt (z.B. den Pfad, Name oder ähnliches). 

Wir starten den Flask Server direkt aus dem Jupyter Notebook mit ```app.run(port="5000")```. Wenn er läuft rufen Sie mit einem Internetbrowser oder Postman die Seite http://127.0.0.1:5000/version auf, da sollten Sie die Ausgabe Ihrer implementierten Funktion sehen. Wenn der Server läuft können Sie ihn mit "Unterbrechen" in VSCode wieder stoppen (davor lässt sich keine andere Juypter Zelle ausführen!).

Implementieren Sie nun die Route ```POST /predict```, die ein Bild entgegen nimmt, es vom Modell interpretieren lässt und die erkannte Ziffer zurück gibt. Denken Sie an die Datenvorverarbeitung ;-).



In [19]:
from flask import Flask
from flask import request 

app = Flask(__name__)

@app.route("/", methods=["GET"])
def get_hello():
    return {"Hello World": "This is the MNIST digit recognition REST-API."}

@app.route("/version", methods=["GET"])
def get_version():
    return {"model_version": model_path}

@app.route("/predict", methods=["POST"])
def post_predict():
    file = request.files["file"]
    predicted_class = predict(file)
    return {"prediction": predicted_class}

app.run(port="5000")


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
ERROR:werkzeug:127.0.0.1 - - [21/Oct/2025 10:35:19] code 400, message Bad request version ('À\x13À')
INFO:werkzeug:127.0.0.1 - - [21/Oct/2025 10:35:19] "\x16\x03\x01\x00å\x01\x00\x00á\x03\x03"»oZO\x82pÇËúïÅ\x81.Éâ·n\x12Ú=Øg'f0¢i\x09¤§ü ³UÒ!à\x8f¸O\x98ù\x9dTê-õ®\x0e0\x8b|'2£È,\x9bê\x98É}*ã\x00$\x13\x01\x13\x02\x13\x03À/À+À0À,À'Ì©Ì¨À\x09À\x13À" 400 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step


INFO:werkzeug:127.0.0.1 - - [21/Oct/2025 10:35:26] "POST /predict HTTP/1.1" 200 -


Um Ihre API zu testen können Sie Postman verwenden (Anmerkung: um ein File als Value mit dem Request mit zu schicken, gibt es auf der rechten Seite von Key ein kleines DropDown zur Auswahl zwischen File und Text):

![Postman Request to Server](imgs/4-Postman.jpg)

### Zugriff auf Web REST-API

Sofern Sie mit den vorherigen Aufgaben durch sind, können Sie versuchen eine REST-API Ihrer Wahl mit Python anzusprechen. Im Jupyter Skript zur Vorlesung finden Sie ein Beispiel wie wir Mistral angesprochen haben. Da Sie für KI-API Endpunkte immer eine Registrierung bzw. API-Key benötigen, ist es ratsam eine API zu nehmen wo sie sowieso einen Acccount haben (z.B. Google, ChatGPT, Mistral, ..). Aus dem Nicht-KI-Umfeld ist es meist einfacher kostenfreie APIs zu bekommen, eine interessante Liste finden Sie hier: https://github.com/public-apis/public-apis